<a href="https://colab.research.google.com/github/kazzttor/estudomei/blob/main/estudo_pandemia_mei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo sobre a pandemia e o empreendedorismo por necessidade

## Parte 1: carregando dados e funcionalidades

In [ ]:
!pip install geopandas unidecode tqdm fastkml
!pip install -U plotly kaleido orca

#Windows VS Code
# %pip install --upgrade numpy pip plotly tqdm pandas geopandas fastkml nbformat "kaleido==0.1.*" orca unidecode shapely matplotlib requests

In [ ]:
import pandas as pd
import requests, io, json
import os
import geopandas as gpd
import plotly.express as px
from tqdm import tqdm
from unidecode import unidecode
import numpy as np
import zipfile
import time
import unicodedata
from shapely.geometry import Point, shape
from fastkml import kml
from shapely.ops import unary_union
from fastkml.kml import Placemark, Folder

In [ ]:
DIR_DADOS = '/content/drive/MyDrive/Colab Notebooks/estudo_mei'

def baixar_arquivos_zip(url_base):
    # Cria a pasta 'dados' se não existir
    dir_dados = DIR_DADOS
    if not os.path.exists(dir_dados):
        os.makedirs(dir_dados)

    # Lista de arquivos a serem baixados
    arquivos = ['Cnaes.zip', 'Motivos.zip', 'Municipios.zip', 'Simples.zip'] + [f'Estabelecimentos{i}.zip' for i in range(10)]

    for arquivo in arquivos:
        url = url_base + arquivo  # Concatena a URL base com o nome do arquivo
        nome_arquivo = os.path.join(dir_dados, arquivo)  # Salvar na pasta 'dados'
        if os.path.exists(nome_arquivo):
            print(f"Arquivo {nome_arquivo} já existe. Pulando download.")
            continue

        # Faz o download do arquivo
        print(f"Baixando {nome_arquivo}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024  # Tamanho do bloco para o tqdm
        with open(nome_arquivo, 'wb') as file:
            for data in tqdm(response.iter_content(block_size), total=total_size // block_size, unit='KB', desc=nome_arquivo):
                file.write(data)

    print("Download concluído.")

def carregar_dados_zip():
    dir_dados = DIR_DADOS
    # Verifica se a pasta 'dados' existe
    if not os.path.exists(dir_dados):
        raise FileNotFoundError("A pasta 'dados' não foi encontrada. Certifique-se de que os arquivos ZIP foram baixados corretamente.")

    arquivos_zip = ['Cnaes.zip', 'Motivos.zip', 'Municipios.zip', 'Simples.zip'] + [f'Estabelecimentos{i}.zip' for i in range(10)]

    # Verifica se os arquivos ZIP existem
    for arquivo in arquivos_zip:
        caminho_arquivo = os.path.join(dir_dados, arquivo)
        if not os.path.exists(caminho_arquivo):
            raise FileNotFoundError(f"O arquivo {arquivo} não foi encontrado na pasta 'dados'. Certifique-se de que o download foi concluído com sucesso.")

    chunk_size = 10000

    # Carregar CNAEs
    print('Carregando CNAES')
    cnaes_df = pd.concat([chunk for chunk in pd.read_csv(f'{dir_dados}/Cnaes.zip', sep=';', header=None, encoding='latin1', names=['codigo', 'descricao'], chunksize=chunk_size, compression='zip', on_bad_lines='skip')], ignore_index=True)

    # Carregar Motivos
    print('Carregando Motivos')
    motivos_df = pd.concat([chunk for chunk in pd.read_csv(f'{dir_dados}/Motivos.zip', sep=';', header=None, encoding='latin1', names=['codigo', 'descricao'], chunksize=chunk_size, compression='zip', on_bad_lines='skip')], ignore_index=True)

    # Carregar Municípios
    print('Carregando Municípios')
    municipios_df = pd.concat([chunk for chunk in pd.read_csv(f'{dir_dados}/Municipios.zip', sep=';', header=None, encoding='latin1', names=['codigo', 'nome'], chunksize=chunk_size, compression='zip', on_bad_lines='skip')], ignore_index=True)

    # Carregar Simples

    simples_chunks = pd.read_csv(f'{dir_dados}/Simples.zip', sep=';', header=None, encoding='latin1',
                                 names=['cnpj_basico', 'opcao_simples', 'data_opcao_simples',
                                        'data_exclusao_simples', 'opcao_mei', 'data_opcao_mei',
                                        'data_exclusao_mei'],
                                 chunksize=chunk_size, compression='zip', on_bad_lines='skip')

    simples_df = pd.concat([chunk for chunk in tqdm(simples_chunks, desc="Carregando Simples", unit="chunk")], ignore_index=True)

    # Carregar Estabelecimentos e mesclar dados
    estabelecimentos_df_list = []
    for i in range(10):
        estabelecimentos_chunks = pd.read_csv(
            f'{dir_dados}/Estabelecimentos{i}.zip', sep=';', header=None, encoding='latin1',
            names=[
                'cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia',
                'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_cidade_exterior',
                'pais', 'data_inicio', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_logradouro',
                'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1',
                'ddd_2', 'telefone_2', 'ddd_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_situacao_especial'
            ],
            chunksize=chunk_size, compression='zip', on_bad_lines='skip'
        )

        for chunk in tqdm(estabelecimentos_chunks, desc=f'Carregando estabelecimentos {i+1}/10', unit='chunk'):
            chunk_filtered = chunk.loc[chunk['identificador_matriz_filial'] == 1, [
                'cnpj_basico', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral',
                'data_inicio', 'cnae_fiscal_principal', 'logradouro', 'numero', 'bairro', 'uf', 'municipio'
            ]]
            estabelecimentos_df_list.append(chunk_filtered)

    estabelecimentos_df = pd.concat(estabelecimentos_df_list, ignore_index=True)

    # Substituir o código de município pelo nome
    print('Anexando Municípios')
    estabelecimentos_df = estabelecimentos_df.merge(municipios_df, left_on='municipio', right_on='codigo', how='left').drop(columns=['municipio', 'codigo']).rename(columns={'nome': 'municipio_nome'})

    # Substituir o código de situação cadastral pelo texto correspondente
    print('Anexando códigos de situação cadastral')
    situacao_map = {1: 'nula', 2: 'ativa', 3: 'suspensa', 4: 'inapta', 8: 'baixada'}
    estabelecimentos_df['situacao_cadastral'] = estabelecimentos_df['situacao_cadastral'].map(situacao_map)

    # Substituir o código do motivo da situação cadastral pelo valor correspondente
    print('Anexando códigos de motivo da situação cadastral')
    estabelecimentos_df = estabelecimentos_df.merge(motivos_df, left_on='motivo_situacao_cadastral', right_on='codigo', how='left').drop(columns=['motivo_situacao_cadastral', 'codigo']).rename(columns={'descricao': 'motivo_situacao_cadastral'})

    # Substituir o código do CNAE fiscal principal pelo valor correspondente
    print('Anexando CNAES')
    estabelecimentos_df = estabelecimentos_df.merge(cnaes_df, left_on='cnae_fiscal_principal', right_on='codigo', how='left').drop(columns=['cnae_fiscal_principal', 'codigo']).rename(columns={'descricao': 'cnae_fiscal_principal'})

    # Vincular os dados do Simples pelo CNPJ BÁSICO
    print('Anexando dados do Simples')
    estabelecimentos_df = estabelecimentos_df.merge(simples_df, on='cnpj_basico', how='left')

    # Filtrar apenas os estabelecimentos com MEI (data_opcao_mei não nula)
    print('Filtrando MEI\'s')
    print(f'Entrada: {estabelecimentos_df.shape[0]} CNPJ\'s totais')
    estabelecimentos_df = estabelecimentos_df[(estabelecimentos_df['data_opcao_mei'].notna()) & (estabelecimentos_df['data_opcao_mei'] != 0)]
    print(f'Saída: {estabelecimentos_df.shape[0]} CNPJ\'s MEI\'s em histórico e atuais.')

    # Converter colunas de data para datetime no formato AAAAMMDD
    print('Padronizando colunas de data')
    estabelecimentos_df['data_situacao_cadastral'] = pd.to_datetime(estabelecimentos_df['data_situacao_cadastral'], format='%Y%m%d', errors='coerce')
    estabelecimentos_df['data_inicio'] = pd.to_datetime(estabelecimentos_df['data_inicio'], format='%Y%m%d', errors='coerce')
    estabelecimentos_df['data_opcao_simples'] = pd.to_datetime(estabelecimentos_df['data_opcao_simples'], format='%Y%m%d', errors='coerce')
    estabelecimentos_df['data_exclusao_simples'] = pd.to_datetime(estabelecimentos_df['data_exclusao_simples'], format='%Y%m%d', errors='coerce')
    estabelecimentos_df['data_opcao_mei'] = pd.to_datetime(estabelecimentos_df['data_opcao_mei'], format='%Y%m%d', errors='coerce')
    estabelecimentos_df['data_exclusao_mei'] = pd.to_datetime(estabelecimentos_df['data_exclusao_mei'], format='%Y%m%d', errors='coerce')

    return estabelecimentos_df

def backup_dataframe(df_estabelecimentos, output=f'{DIR_DADOS}/estabelecimentos_mei.zip'):
    chunk_size = 10000
    if not os.path.exists('dados'):
        os.makedirs('dados')
    raiz = os.path.splitext(output)
    csv_file = f'{raiz[0]}.csv'

    # Exportar dataframe para CSV em chunks
    chunks = np.array_split(df_estabelecimentos.index, chunk_size)
    for chunck, subset in enumerate(tqdm(chunks, desc='Exportando dataframe', unit='chunk')):
        if chunck == 0:  # primeira linha
            df_estabelecimentos.loc[subset].to_csv(csv_file, mode='w', index=True)
        else:
            df_estabelecimentos.loc[subset].to_csv(csv_file, header=None, mode='a', index=True)
    print('Backup do dataframe realizado com sucesso!')

    # Compactar o arquivo CSV em um arquivo ZIP
    with zipfile.ZipFile(output, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(csv_file, os.path.basename(csv_file))

    # Verificar se a compressão foi bem-sucedida e remover o arquivo CSV original
    if os.path.exists(output):
        os.remove(csv_file)
        print(f'Compressão bem-sucedida! Arquivo {csv_file} removido.')
    else:
        print(f'Compressão falhou! Arquivo {csv_file} não removido.')

def restore_dataframe(arquivo, filtros=None):
    if not os.path.exists(arquivo):
        raise FileNotFoundError(f"O arquivo {arquivo} não foi encontrado. Certifique-se de que o arquivo exista e que o local do arquivo foi informado corretamente.")

    print(f'Restaurando dataframe a partir do arquivo {arquivo}')
    chunk_size = 10000

    # Definir os tipos de dados das colunas, exceto as descartadas (índice e número)
    dtypes = {
        'cnpj_basico': 'Int64',
        'latitude': 'float64',
        'longitude': 'float64'
    }

    parse_dates = [
        'data_situacao_cadastral',
        'data_inicio',
        'data_opcao_simples',
        'data_exclusao_simples',
        'data_opcao_mei',
        'data_exclusao_mei'
    ]

    # Verificar as colunas existentes e descartar as que não serão usadas
    first_chunk = pd.read_csv(arquivo, nrows=1, compression='zip')
    columns_to_use = [col for col in first_chunk.columns if col not in ['Unnamed: 0', 'numero']]

    dtypes = {col: dtype for col, dtype in dtypes.items() if col in columns_to_use}
    parse_dates = [col for col in parse_dates if col in columns_to_use]

    # Ler o arquivo em chunks, descartando as colunas 'Unnamed: 0' e 'numero'
    chunks = pd.read_csv(
        arquivo,
        chunksize=chunk_size,
        compression='zip',
        on_bad_lines='skip',
        dtype=dtypes,
        parse_dates=parse_dates,
        usecols=columns_to_use
    )

    # Filtrar e concatenar os chunks
    df_restored = pd.concat(
        [chunk.query(filtros) if filtros else chunk for chunk in tqdm(chunks, desc="Carregando Dataframe", unit="chunk")],
        ignore_index=True
    )

    print('Dataframe restaurado com sucesso!')
    return df_restored

def plotar_mapa_estabelecimentos(df_estabelecimentos, cidade, nome_arquivo_mapa=None, titulo=None, nota=None):
    print('Gerando mapa, Aguarde...')
    df_com_coordenadas = df_estabelecimentos.dropna(subset=['latitude', 'longitude'])
    if titulo is None:
        titulo = f"Mapa de Estabelecimentos MEI em {cidade}"
    if nota is not None:
        notarodape = f"{nota}<br>Fonte: Receita Federal do Brasil e IBGE. Elaborado pelo autor."
    else:
        notarodape = "Fonte: Receita Federal do Brasil e IBGE. Elaborado pelo autor."

    gdf_com_coordenadas = gpd.GeoDataFrame(df_com_coordenadas, geometry=gpd.points_from_xy(df_com_coordenadas.longitude, df_com_coordenadas.latitude))
    df_agrupado = gdf_com_coordenadas.groupby('bairro')['geometry'].apply(lambda x: x.unary_union.centroid).reset_index()
    df_agrupado['latitude'] = df_agrupado['geometry'].apply(lambda x: x.y)
    df_agrupado['longitude'] = df_agrupado['geometry'].apply(lambda x: x.x)

    total_estabelecimentos_bairro = df_estabelecimentos.groupby('bairro').size().reset_index(name='count')
    df_agrupado = pd.merge(df_agrupado, total_estabelecimentos_bairro, on='bairro')

    if cidade.lower() == 'são paulo':
        df_agrupado = pd.merge(df_agrupado, df_com_coordenadas[['bairro', 'centro_expandido']].drop_duplicates(), on='bairro', how='left')
        df_agrupado['color'] = df_agrupado['centro_expandido'].apply(lambda x: 'blue' if x == 'S' else 'red')
        df_agrupado['color_label'] = df_agrupado['centro_expandido'].apply(lambda x: 'Centro Expandido' if x == 'S' else 'Periferia')
        color_column = 'color_label'
    else:
        color_column = None

    centro_lat = df_com_coordenadas['latitude'].mean()
    centro_lon = df_com_coordenadas['longitude'].mean()
    geodf = ibge_municipio_geodataframe(ibge_id_municipio(slugify(cidade))[0])
    bounds = geodf.total_bounds
    centro_municipio = geodf.geometry.centroid.iloc[0]

    def calcular_zoom(bounds):
        minx, miny, maxx, maxy = bounds
        x_diff = maxx - minx
        y_diff = maxy - miny
        max_diff = max(x_diff, y_diff)
        zoom = 12 - (max_diff/0.25)
        return zoom

    nivel_zoom = calcular_zoom(bounds)

    fig = px.scatter_mapbox(df_agrupado,
                            lat='latitude',
                            lon='longitude',
                            size='count',
                            hover_name='bairro',
                            title=titulo,
                            labels={'count': 'Quantidade de Estabelecimentos', 'color_label': 'Região'},
                            center={"lat": centro_lat, "lon": centro_lon},
                            color=color_column)

    fig.update_layout(
        mapbox={
            "style": "carto-positron",
            "zoom": nivel_zoom,
            "center":{"lat":centro_municipio.y, "lon":centro_municipio.x},
            "layers": [
                {
                    "source": json.loads(geodf.geometry.to_json()),
                    "below": "traces",
                    "type": "line",
                    "color": "black",
                    "line": {"width": 1.5},
                }
            ],
        },
        legend_title_text='Região' if cidade.lower() == 'são paulo' else None,
        margin={"l": 0, "r": 0, "t": 50, "b": 50},
        width=720,
        height=800,
        annotations=[
            dict(
                text=notarodape,
                x=0, y=0.02,
                xref='paper', yref='paper',
                showarrow=False,
                xanchor='left',
                bgcolor='rgba(255, 255, 255, 0.8)'
            )
        ]
    )

    if nome_arquivo_mapa:
        try:
            fig.write_image(f"{DIR_DADOS}/{nome_arquivo_mapa}.png")
            print(f"Mapa salvo como '{DIR_DADOS}/{nome_arquivo_mapa}.png'")
        except ValueError as e:
            if 'kaleido' in str(e):
                print("Falha ao utilizar Kaleido, tentando com Orca...")
                fig.write_image(f"{DIR_DADOS}/{nome_arquivo_mapa}.png", engine="orca")
                print(f"Mapa salvo como '{DIR_DADOS}/{nome_arquivo_mapa}.png' com Orca")

    return fig

def slugify(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = text.replace(' ', '-')
    text = ''.join(c for c in text if c.isalnum() or c in ['-', '_'])
    return text.lower()

def ibge_id_municipio(municipio_slug):
    """
    Busca o ID do município no IBGE usando o slug do município.

    Args:
        municipio_slug (str): Slug do município (nome formatado para URL).

    Returns:
        tuple: ID do município e nome do município.
    """
    base_url = f'https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{municipio_slug}'
    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()
        # Verifica se a resposta contém os dados esperados
        if 'id' in data and 'nome' in data:
            return data['id'], data['nome']
        else:
            raise ValueError(f"Formato de dados inesperado da API: {data}")
    else:
        raise Exception(f'Erro ao buscar ID do município: {response.status_code}')

def ibge_municipio_geodataframe(id_municipio):
    """
    Busca os dados geográficos do município no IBGE e retorna um GeoDataFrame.

    Args:
        id_municipio (int): ID do município.

    Returns:
        GeoDataFrame: Dados geográficos do município.
    """
    url = f'https://servicodados.ibge.gov.br/api/v3/malhas/municipios/{id_municipio}?formato=application/vnd.geo+json'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return gpd.GeoDataFrame.from_features(data['features'])
    else:
        raise Exception(f'Erro ao obter dados geográficos do município: {response.status_code}')

def download_kml(url, dir_dados, filename):
  """Downloads a KML file from a given URL and saves it to a specified directory."""

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    if not os.path.exists(dir_dados):
      os.makedirs(dir_dados)

    filepath = os.path.join(dir_dados, filename)
    with open(filepath, 'wb') as f:
      f.write(response.content)

    print(f"KML file downloaded successfully to: {filepath}")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading KML file: {e}")

def extrair_bairros_kml(kml_file, df_enderecos):
    """
    Extrai bairros contidos dentro de uma área delimitada por um arquivo KML.

    Args:
        kml_file (str): Caminho para o arquivo KML.
        df_enderecos (pd.DataFrame): DataFrame com dados de endereços e coordenadas geográficas.

    Returns:
        pd.DataFrame: DataFrame com os bairros contidos na área delimitada.
    """
    # Carregar o arquivo KML como bytes
    # with open(kml_file, 'rt') as f: # Change 'rt' to 'rb' to read in binary mode
    #     doc = f.read()

    # k_data = kml.KML()
    # k_data.from_string(doc) # Now 'doc' is a bytes string

    k_data = kml.KML.parse(kml_file)

    # Extrair a geometria do KML
    k_features = k_data.features
    placemarks = list(k_features[0].features)
    polygons = [shape(placemark.geometry) for placemark in placemarks]

    # Converter DataFrame de endereços para GeoDataFrame
    gdf_enderecos = gpd.GeoDataFrame(
        df_enderecos,
        geometry=gpd.points_from_xy(df_enderecos.longitude, df_enderecos.latitude)
    )

    # Verificar quais pontos estão dentro dos polígonos do KML
    bairros_contidos = gdf_enderecos[gdf_enderecos.geometry.apply(lambda x: any(polygon.contains(x) for polygon in polygons))]

    return bairros_contidos

def logradouros_IBGE(municipio: str, uf: str) -> pd.DataFrame:
    nome_arquivo = f'{DIR_DADOS}/df_{slugify(municipio)}_{slugify(uf)}_logradouros.zip'
    if os.path.exists(nome_arquivo):
        print(f'Restaurando dataframe a partir do arquivo local {nome_arquivo}')
        return restore_dataframe(nome_arquivo)
    else:
        print('Arquivo local não encontrado. Baixando dados do IBGE')
        codigos_uf = {
            "AC": "12", "AL": "27", "AP": "16", "AM": "13", "BA": "29", "CE": "23", "DF": "53",
            "ES": "32", "GO": "52", "MA": "21", "MT": "51", "MS": "50", "MG": "31", "PA": "15",
            "PB": "25", "PR": "41", "PE": "26", "PI": "22", "RJ": "33", "RN": "24", "RS": "43",
            "RO": "11", "RR": "14", "SC": "42", "SP": "35", "SE": "28", "TO": "17"
        }

        codigo_uf = codigos_uf.get(uf.upper())
        if not codigo_uf:
            raise ValueError("UF inválida ou não encontrada.")

        municipio_slug = slugify(municipio)
        municipio_id, municipio_nome = ibge_id_municipio(municipio_slug)
        municipio_nome_formatado = unicodedata.normalize('NFKD', municipio_nome).encode('ascii', 'ignore').decode('utf-8').replace(" ", "_").upper()
        url = f"https://ftp.ibge.gov.br/Cadastro_Nacional_de_Enderecos_para_Fins_Estatisticos/Censo_Demografico_2022/Arquivos_CNEFE/CSV/Municipio/{codigo_uf}_{uf.upper()}/{municipio_id}_{municipio_nome_formatado}.zip"

        tentativas = 3
        file_path = f"{DIR_DADOS}/{municipio_id}_{municipio_nome_formatado}.zip"

        while tentativas > 0:
            try:
                headers = {"Range": f"bytes={os.path.getsize(file_path)}-"} if os.path.exists(file_path) else {}
                with requests.get(url, headers=headers, stream=True) as response:
                    total_size = int(response.headers.get('content-length', 0))
                    mode = 'ab' if os.path.exists(file_path) else 'wb'
                    with open(file_path, mode) as file, tqdm(
                        total=total_size, unit='B', unit_scale=True, desc=f"Baixando dados de {municipio} - {uf}"
                    ) as pbar:
                        for data in response.iter_content(chunk_size=1024):
                            file.write(data)
                            pbar.update(len(data))
                break  # Sucesso, sair do loop
            except requests.exceptions.RequestException as e:
                print(f"Erro no download: {e}. Tentando novamente...")
                tentativas -= 1
                if tentativas == 0:
                    raise Exception("Falha ao baixar os dados após 3 tentativas.")

        with open(file_path, 'rb') as file:
            with io.BytesIO(file.read()) as file_io:
                colunas_selecionadas = ['DSC_LOCALIDADE', 'NOM_SEGLOGR', 'LATITUDE', 'LONGITUDE']
                df = pd.read_csv(file_io, compression='zip', sep=';', encoding='latin-1', usecols=colunas_selecionadas).reset_index()
                df = df.rename(columns={
                    'NOM_SEGLOGR': 'logradouro',
                    'DSC_LOCALIDADE': 'bairro',
                    'LATITUDE': 'latitude',
                    'LONGITUDE': 'longitude'
                })
                df = df[['logradouro', 'bairro', 'latitude', 'longitude']]
                df = df.dropna(subset=['latitude', 'longitude'])
                df = df.groupby(['logradouro', 'bairro'])[['latitude', 'longitude']].mean().reset_index()
        backup_dataframe(df, output=nome_arquivo)
        return df

def fase1 ():
  # A fase 1 é acionada quando não há nenhum conjunto de dados. Consiste em baixar os dados da receita federal e compilá-los em um dataframe.
  url_rfb = 'https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2024-09/'
  baixar_arquivos_zip(url_rfb)
  df_estabelecimentos = carregar_dados_zip()
  backup_dataframe(df_estabelecimentos)
  return df_estabelecimentos

def fase2 (arquivo, municipio, uf, modo='df'):
  #A fase 2 é acionada quando há os dados de estabelecimentos salvos localmente e compilados para o estudo.
  municipio_u = unicodedata.normalize('NFKD', municipio).encode('ascii', 'ignore').decode('utf-8').upper()
  uf_u = uf.upper()
  if modo == 'df':
    df_estabelecimentos = arquivo.loc[(arquivo['municipio_nome'] == municipio_u) & (arquivo['uf'] == uf_u)]
  elif modo == 'restore':
    df_estabelecimentos = restore_dataframe(arquivo, filtros=f"municipio_nome == '{municipio_u}' & uf == '{uf_u}'")
  else:
    raise ValueError("Modo inválido. Use 'df' ou 'restore'.")
  locais_sp = logradouros_IBGE(municipio, uf)
  fase2 = pd.merge(df_estabelecimentos, locais_sp, on=['logradouro', 'bairro'], how='inner')
  arq_fase2 = f'{DIR_DADOS}/df_{slugify(municipio)}_fase2.zip'
  backup_dataframe(fase2, output=arq_fase2)
  return fase2

def fase3(arq_cidade, cidade, modo="df"):
    if modo == "df":
        df_cidade = arq_cidade
    elif modo == "restore":
        df_cidade = restore_dataframe(arq_cidade)
    else:
        raise ValueError("Modo inválido. Use 'df' ou 'restore'.")

    if cidade == 'São Paulo':
        print("Rotina especial para a cidade de São Paulo")
        df_com_coordenadas = df_cidade.dropna(subset=['latitude', 'longitude'])

        # Criar GeoDataFrame a partir do DataFrame original
        geometry = [Point(xy) for xy in zip(df_com_coordenadas.longitude, df_com_coordenadas.latitude)]
        gdf = gpd.GeoDataFrame(df_com_coordenadas, geometry=geometry)

        # Agrupar por bairro e calcular o centroide
        gdf_centroides = gdf.groupby('bairro').geometry.apply(lambda x: x.union_all().centroid).reset_index()
        gdf_centroides.columns = ['bairro', 'centroide']

        # Separar latitude e longitude do centroide
        gdf_centroides['latitude'] = gdf_centroides.centroide.apply(lambda p: p.y)
        gdf_centroides['longitude'] = gdf_centroides.centroide.apply(lambda p: p.x)
        gdf_centroides = gdf_centroides.drop(columns='centroide')

        bairros_contidos = extrair_bairros_kml(f'{DIR_DADOS}/rodizio_sp.kml', gdf_centroides)
        df_cidade['centro_expandido'] = 'N'  # Inicializa a coluna com 'N'
        df_cidade.loc[df_cidade['bairro'].isin(bairros_contidos['bairro']), 'centro_expandido'] = 'S'

    df_cidade['periodo'] = 'outro'
    df_cidade.loc[(df_cidade['data_inicio'] >= '2018-02-03') & (df_cidade['data_inicio'] <= '2020-02-02'), 'periodo'] = 'antes'
    df_cidade.loc[(df_cidade['data_inicio'] >= '2020-02-03') & (df_cidade['data_inicio'] <= '2022-05-22'), 'periodo'] = 'pandemia'
    df_cidade.loc[(df_cidade['data_inicio'] >= '2022-05-23') & (df_cidade['data_inicio'] <= '2024-05-22'), 'periodo'] = 'depois'
    df_cidade = df_cidade[df_cidade['periodo'] != 'outro']

    arq_fase3 = f'{DIR_DADOS}/df_{slugify(cidade)}_fase3.zip'
    backup_dataframe(df_cidade, output=arq_fase3)
    return df_cidade

def gerar_df_cidade(cidade, uf):
    arquivo_df = f'{DIR_DADOS}/estabelecimentos_mei.zip'
    arqfase2 = f'{DIR_DADOS}/df_{slugify(cidade)}_fase2.zip'
    arqfase3 = f'{DIR_DADOS}/df_{slugify(cidade)}_fase3.zip'
    fase2_df = pd.DataFrame()
    fase3_df = pd.DataFrame()
    if os.path.exists(arqfase3):
        print(f'Fase 3 de {cidade} encontrada em {arqfase3}')
        fase3_df = restore_dataframe(arqfase3)
        print(f'Total de registros (fase 3): {fase3_df.shape[0]}')
    else:
        if os.path.exists(arqfase2):
            print(f'Fase 3 de {cidade} não encontrada, mas encontrada Fase 2 em {arqfase2}')
            fase3_df = fase3(arqfase2, cidade, modo='restore')
            print(f'Total de registros (fase 3): {fase3_df.shape[0]}')
        else:
            if os.path.exists(arquivo_df):
                print(f'Fase 3 e 2 de {cidade} não encontradas, mas encontrada Fase 1 em {arquivo_df}')
                fase2_df = fase2(arquivo_df, cidade, uf, modo='restore')
                print(f'Total de registros (fase 2): {fase2_df.shape[0]}')
                fase3_df = fase3(fase2_df, cidade)
                print(f'Total de registros (fase 3): {fase3_df.shape[0]}')
            else:
                print(f'Não concontrados arquivos de dados, gerando dataframes para {cidade}')
                df_estabelecimentos = fase1()
                print(f'Total de registros (fase 1): {df_estabelecimentos.shape[0]}')
                fase2_df = fase2(df_estabelecimentos, cidade, uf)
                print(f'Total de registros (fase 2): {fase2_df.shape[0]}')
                fase3_df = fase3(fase2_df, cidade)
                print(f'Total de registros (fase 3): {fase3_df.shape[0]}')
    return fase3_df

## Fase 2: carregar dataframes do estudo

In [ ]:
def gerar_df_diadema():
  cidade = 'Diadema'
  uf = 'SP'
  df_diadema = gerar_df_cidade(cidade, uf)
  df_diadema.info()
  return df_diadema

In [ ]:
def gerar_df_saopaulo():
  kml_url = "https://www.google.com/maps/d/kml?mid=1Xg6ACUNSiuS0IOr7p7t9c6IvLx4&resourcekey&lid=z9TqTcegPvdk.kkQKI570N-uE&forcekml=1"
  dir_dados = DIR_DADOS
  filename = "rodizio_sp.kml"

  download_kml(kml_url, dir_dados, filename)

  cidade = 'São Paulo'
  uf = 'SP'
  df_sp = gerar_df_cidade(cidade, uf)
  df_sp.info()
  return df_sp

In [ ]:
def gerar_df_bsb():
  cidade = 'Brasília'
  uf = 'DF'
  df_bsb = gerar_df_cidade(cidade, uf)
  df_bsb.info()
  return df_bsb

In [ ]:
def gerar_df_rjo():
  cidade = 'Rio de Janeiro'
  uf = 'RJ'
  df_rjo = gerar_df_cidade(cidade, uf)
  df_rjo.info()
  return df_rjo

## Fase 3: estudo sobre a cidade de São Paulo

### Datas marco

Emergência sanitária no Brasil:

- **Início**: 03/02/2020
- **Fim**: 22/05/2022

Observação: 03/02/2018 a 02/02/2020 (antes) e 23/05/2022 a 22/05/2024 (depois)

Antes:

- Centro expandido: 44749 (25,9 %)
- Fora do Centro: 127539 (74,1 %)

Pandemia:

- Centro expandido: 44981 (20,7 %)
- Fora do Centro: 181697 (79,3 %)

Depois:

- Centro expandido: 35431 (19,2 %)
- Fora do Centro: 148329 (80,8 %)


In [ ]:
df_merged = gerar_df_saopaulo()

In [ ]:
df_merged['centro_expandido'].value_counts()

In [ ]:
fig = plotar_mapa_estabelecimentos(df_merged, 'São Paulo',nome_arquivo_mapa='mapa_mei_sp', nota='Estabelecimentos MEI criados na cidade de São Paulo entre 03/02/2018 e 22/05/2024.')
fig.show()

### Análise exploratória São Paulo

In [ ]:
df_merged.loc[df_merged['periodo']=='pandemia'][['centro_expandido','situacao_cadastral']].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['situacao_cadastral']=='ativa')][['centro_expandido','opcao_mei']].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='S') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='N') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='S') & (df_merged['opcao_mei']=='S') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='N') & (df_merged['opcao_mei']=='S') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='N') & (df_merged['opcao_mei']=='N') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='S') & (df_merged['opcao_mei']=='N') & (df_merged['situacao_cadastral']=='ativa')]['cnae_fiscal_principal'].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='S')][['situacao_cadastral','motivo_situacao_cadastral']].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['centro_expandido']=='N')][['situacao_cadastral','motivo_situacao_cadastral']].value_counts()

In [ ]:
df_merged.loc[(df_merged['periodo']=='pandemia') & (df_merged['situacao_cadastral']=='baixada')][['motivo_situacao_cadastral','cnae_fiscal_principal']].value_counts()

### Gráficos Temporais

In [ ]:
import pandas as pd
import plotly.express as px

# Converta a coluna 'data_inicio' para o tipo datetime
df_merged['data_inicio'] = pd.to_datetime(df_merged['data_inicio'])

# Agrupe os dados mensalmente pela soma de registros, divididos pela coluna 'centro_expandido'
df_agrupado = df_merged.groupby([pd.Grouper(key='data_inicio', freq='ME'), 'centro_expandido']).size().reset_index(name='total_registros')

# Crie um gráfico de linha mensal
fig = px.line(df_agrupado,
              x='data_inicio',
              y='total_registros',
              color='centro_expandido',
              title='Número de Registros por Mês e Centro Expandido',
              labels={'total_registros': 'Número de Registros', 'data_inicio': 'Mês', 'centro_expandido': 'Localização'},
              color_discrete_map={'S': 'blue', 'N': 'red'})

# Atualize a legenda
fig.for_each_trace(lambda t: t.update(name={'S': 'Centro Expandido', 'N': 'Periferia'}[t.name]))

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")

# Exiba o gráfico
fig.show()


In [ ]:
# prompt: Vamos fazer o detalhamento do gráfico anterior, em vez do saldo, vamos discriminar o total de cada situação cadastral, por localização (centro expandido e periferia) em dois gráficos de linha, um pra centro expandido e outro e outro para pefiferia.

# Converta a coluna 'data_situacao_cadastral' para o tipo datetime
df_merged['data_situacao_cadastral'] = pd.to_datetime(df_merged['data_situacao_cadastral'])

# Agrupe os dados mensalmente pelo número de registros por cada situação cadastral, divididos pela coluna 'centro_expandido'
df_agrupado = df_merged.groupby([pd.Grouper(key='data_situacao_cadastral', freq='ME'), 'centro_expandido', 'situacao_cadastral']).size().reset_index(name='total_registros')

# Crie um gráfico de linha mensal para o Centro Expandido
df_centro_expandido = df_agrupado[df_agrupado['centro_expandido'] == 'S']
fig2_centro_expandido = px.line(df_centro_expandido,
                             x='data_situacao_cadastral',
                             y='total_registros',
                             color='situacao_cadastral',
                             title='Total de Situações Cadastrais por Mês (Centro Expandido)',
                             labels={'total_registros': 'Número de Registros', 'data_situacao_cadastral': 'Mês', 'situacao_cadastral': 'Situação Cadastral'})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig2_centro_expandido.add_vrect(x0='2020-02-03', x1='2022-05-22',
                            fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
                            annotation_text="Pandemia", annotation_position="top left")

fig2_centro_expandido.add_vrect(x0='2018-02-03', x1='2020-02-02',
                            fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
                            annotation_text="Pré-Pandemia", annotation_position="top left")

fig2_centro_expandido.add_vrect(x0='2022-05-23', x1='2024-05-22',
                            fillcolor="pink", opacity=0.2, layer="below", line_width=0,
                            annotation_text="Pós-Pandemia", annotation_position="top left")

fig2_centro_expandido.show()


# Crie um gráfico de linha mensal para a Periferia
df_periferia = df_agrupado[df_agrupado['centro_expandido'] == 'N']
fig2_periferia = px.line(df_periferia,
                        x='data_situacao_cadastral',
                        y='total_registros',
                        color='situacao_cadastral',
                        title='Total de Situações Cadastrais por Mês (Periferia)',
                        labels={'total_registros': 'Número de Registros', 'data_situacao_cadastral': 'Mês', 'situacao_cadastral': 'Situação Cadastral'})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig2_periferia.add_vrect(x0='2020-02-03', x1='2022-05-22',
                        fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pandemia", annotation_position="top left")

fig2_periferia.add_vrect(x0='2018-02-03', x1='2020-02-02',
                        fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pré-Pandemia", annotation_position="top left")

fig2_periferia.add_vrect(x0='2022-05-23', x1='2024-05-22',
                        fillcolor="pink", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pós-Pandemia", annotation_position="top left")

fig2_periferia.show()

In [ ]:
# prompt: Vamos fazer o mesmo gráfico da célula anterior, porém em vez do saldo mensal, ser o saldo mensal acumulado.

# Converta a coluna 'data_situacao_cadastral' para o tipo datetime
df_merged['data_situacao_cadastral'] = pd.to_datetime(df_merged['data_situacao_cadastral'])

# Crie uma nova coluna 'saldo_mei_ativos' que representa a diferença entre o número de MEIs ativos e os demais
df_merged['saldo_mei_ativos'] = 0  # Inicializa a coluna com 0
df_merged.loc[df_merged['situacao_cadastral'] == 'ativa', 'saldo_mei_ativos'] = 1  # Se a situação cadastral é 'ativa', o saldo é 1
df_merged.loc[df_merged['situacao_cadastral'] != 'ativa', 'saldo_mei_ativos'] = -1  # Se não, o saldo é -1

# Agrupe os dados mensalmente pela soma de registros, divididos pela coluna 'centro_expandido'
df_agrupado = df_merged.groupby([pd.Grouper(key='data_situacao_cadastral', freq='ME'), 'centro_expandido'])['saldo_mei_ativos'].sum().reset_index(name='saldo_mei_ativos')

# Calcule o saldo acumulado
df_agrupado['saldo_acumulado'] = df_agrupado.groupby('centro_expandido')['saldo_mei_ativos'].cumsum()

# Crie um gráfico de linha mensal com o saldo acumulado
fig = px.area(df_agrupado,
              x='data_situacao_cadastral',
              y='saldo_acumulado',
              color='centro_expandido',
              title='Saldo Acumulado de MEIs Ativos por Mês e Centro Expandido',
              labels={'saldo_acumulado': 'Saldo Acumulado de MEIs Ativos', 'data_situacao_cadastral': 'Mês', 'centro_expandido': 'Localização'},
              color_discrete_map={'S': 'blue', 'N': 'red'})

# Atualize a legenda
fig.for_each_trace(lambda t: t.update(name={'S': 'Centro Expandido', 'N': 'Periferia'}[t.name]))

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")

# Exiba o gráfico
fig.show()

In [ ]:
!pip install plotly-orca

In [ ]:
# prompt: a partir do dataframe df_sp, faça um gráfico de linha usando a da coluna data_inicio agrupado por mês, bairro, se o bairro é do centro expandido, ou não, e a quantidade de MEI's criados. use o fundo para separar os períodos, antes, pandemia e depois. Separe os bairros do centro expandido como linhas de tons azuis e fora como linhas de tons vermelhos.
df_sp = df_merged.copy()
# Converta a coluna 'data_inicio' para o tipo datetime
df_sp['data_inicio'] = pd.to_datetime(df_sp['data_inicio'])

# Agrupe os dados por mês, bairro, centro expandido e conte a quantidade de MEIs
df_agrupado = df_sp.groupby([pd.Grouper(key='data_inicio', freq='ME'), 'bairro', 'centro_expandido'])['cnpj_basico'].count().reset_index(name='total_mei')

# Crie um gráfico de linha
fig = px.line(df_agrupado,
              x='data_inicio',
              y='total_mei',
              color='bairro',
              facet_row='centro_expandido',
              title='Quantidade de MEIs criados por mês, bairro e localização (Centro Expandido/Fora)',
              labels={'data_inicio': 'Mês', 'total_mei': 'Quantidade de MEIs', 'bairro': 'Bairro', 'centro_expandido': 'Centro Expandido'},
              color_discrete_map={'S': ['blue', 'royalblue', 'darkblue'],
                                 'N': ['red', 'firebrick', 'maroon']})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")

fig.show()

In [ ]:
# prompt: vamos criar uma função que gere a série temporal a partir de um dataframe, conforme modelo já testado, usando como variáveis o dataframe, a coluna temporal de referência, o valor a ser contado, as categorias (as linhas que seriam mensuradas), e plote o gráfico de linha, com as separações de épocas (antes, pandemia e depois)

def gerar_serie_temporal_e_plotar(df, coluna_temporal, valor_a_contar, categorias, titulo_grafico):
  """
  Gera uma série temporal a partir de um DataFrame, agrupando por mês e categoria,
  e plota um gráfico de linha com as separações de épocas (antes, pandemia, depois).

  Args:
      df: DataFrame com os dados.
      coluna_temporal: Nome da coluna com a data a ser usada na série temporal.
      valor_a_contar: Nome da coluna com o valor a ser contado.
      categorias: Lista de nomes de colunas para agrupar os dados.
      titulo_grafico: Título do gráfico.
  """

  # Converter a coluna temporal para datetime
  df[coluna_temporal] = pd.to_datetime(df[coluna_temporal])

  # Agrupar os dados por mês e categorias, e contar o valor a ser contado
  df_agrupado = df.groupby([pd.Grouper(key=coluna_temporal, freq='M')] + categorias)[valor_a_contar].count().reset_index()

  # Criar o gráfico de linha
  fig = px.line(df_agrupado,
                x=coluna_temporal,
                y=valor_a_contar,
                color=categorias[0] if categorias else None,
                title=titulo_grafico,
                labels={valor_a_contar: 'Quantidade', coluna_temporal: 'Mês'})

  # Adicionar áreas de fundo para as épocas
  fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
               fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
               annotation_text="Pandemia", annotation_position="top left")

  fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
               fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
               annotation_text="Pré-Pandemia", annotation_position="top left")

  fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
               fillcolor="pink", opacity=0.2, layer="below", line_width=0,
               annotation_text="Pós-Pandemia", annotation_position="top left")

  fig.show()

### Nível de emprego IBGE

In [ ]:
import requests
import pandas as pd

# URL da API do IBGE
url = "https://servicodados.ibge.gov.br/api/v3/agregados/6379/periodos/201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/4097?localidades=N1[all]"

# Fazer a requisição para a API
response = requests.get(url)
data = response.json()

# Extração dos dados
serie_temporal = data[0]['resultados'][0]['series'][0]['serie']

# Transformação dos dados em um DataFrame
df = pd.DataFrame(serie_temporal.items(), columns=['mesano', 'nivel_ocupacao'])

# Conversão da coluna 'mesano' para o formato de data YYYYMM
df['mesano'] = pd.to_datetime(df['mesano'], format='%Y%m')
df['nivel_ocupacao'] = df['nivel_ocupacao'].astype(float).round(1)
# Exibir o DataFrame resultante
print(df)


In [ ]:
import plotly.express as px
import pandas as pd

# Criar o gráfico
fig = px.line(df,
              x='mesano',
              y='nivel_ocupacao',
              title='Nível de ocupação do Brasil*',
              labels={'nivel_ocupacao': 'Nível de ocupação (%)', 'mesano': 'Mês'})

# Adicionar áreas de fundo para as épocas
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")

# Adicionar uma anotação no rodapé do gráfico
fig.add_annotation(
    text='Percentual de pessoas com 14 anos ou mais ocupadas. Fonte: IBGE.',
    xref='paper', yref='paper',
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=12)
)

fig.show()

In [ ]:
import pandas as pd

# Supondo que df já esteja definido
df['mesano'] = pd.to_datetime(df['mesano'], format='%Y%m')

# Agregar os dados por ano
df['ano'] = df['mesano'].dt.year
df_ano_a_ano = df.groupby('ano')['nivel_ocupacao'].mean().reset_index()

# Exibir a tabela resultante
print(df_ano_a_ano)


### Dados de tempo de sobrevivência e inaptidão de MEI's

In [ ]:
# prompt: crie para o dataset df_sp uma coluna chamada meses_duracao para registros com a situacao_cadastral = baixada como a diferença em meses entre data_situacao_cadastral e data_inicio em meses, podemdo ser fracionado com uma cada decimal. Mantenha os registros não afetados como células nulas.

# Crie a coluna 'meses_duracao' com valores nulos
df_sp['meses_duracao'] = np.nan

# Calcule a diferença em meses apenas para registros com 'situacao_cadastral' igual a 'baixada'
baixadas = df_sp['situacao_cadastral'] == 'baixada'
df_sp.loc[baixadas, 'meses_duracao'] = (df_sp.loc[baixadas, 'data_situacao_cadastral'] - df_sp.loc[baixadas, 'data_inicio']) / pd.Timedelta(days=30.44)

In [ ]:
# prompt: crie para o dataset df_sp uma coluna chamada meses_abandono para registros com a situacao_cadastral = inapta como a diferença em meses entre data_situacao_cadastral e data_inicio em meses, podemdo ser fracionado com uma cada decimal. Mantenha os registros não afetados como células nulas.

# Crie a coluna 'meses_abandono' com valores nulos
df_sp['meses_abandono'] = np.nan

# Calcule a diferença em meses apenas para registros com 'situacao_cadastral' igual a 'inapta'
inaptas = df_sp['situacao_cadastral'] == 'inapta'
df_sp.loc[inaptas, 'meses_abandono'] = (df_sp.loc[inaptas, 'data_situacao_cadastral'] - df_sp.loc[inaptas, 'data_inicio']) / pd.Timedelta(days=30.44)

In [ ]:
# prompt: calcule o valor médio de meses_duracao e de meses_abandono  e quantidade de registros com meses_duracao e meses_abandono para valores não-nulos do dataframe df_sp agrupados pelas colunas periodo e centro_expandido. Calcule o percentual de registros em relação ao total de registros por grupo (com valores meses_duracao, meses_abandono e nulos)

# Group data by 'periodo' and 'centro_expandido'
grouped = df_sp.groupby(['periodo', 'centro_expandido'])

# Calculate the mean of 'meses_duracao' and 'meses_abandono', and the count of non-null values
result = grouped[['meses_duracao', 'meses_abandono']].agg(['mean', 'count'])

# Rename columns for clarity
result.columns = ['tempo_medio_baixa', 'empresas_baixadas', 'tempo_medio_inaptidao', 'empresas_inaptas']

# Calculate the total count of records for each group (including null values)
total_counts = grouped.size().rename('total_registros')

# Combine the results with the total counts
result = pd.concat([result, total_counts], axis=1)

# Calculate the percentage of records
result['percentual_baixadas'] = (result['empresas_baixadas'] / result['total_registros']) * 100
result['percentual_inaptas'] = (result['empresas_inaptas'] / result['total_registros']) * 100

result

In [ ]:
# prompt: crie uma coluna chamada desenquadrado com valores S ou N onde o valor de  data_exclusao_mei seja não-nulo e maior que data_opcao_mei para o dataframe df_sp

# Crie a coluna 'desenquadrado' com valores 'N' inicialmente
df_sp['desenquadrado'] = 'N'

# Atualize os valores para 'S' onde a condição é satisfeita
df_sp.loc[(df_sp['data_exclusao_mei'].notnull()) & (df_sp['data_exclusao_mei'] > df_sp['data_opcao_mei']), 'desenquadrado'] = 'S'

In [ ]:
# prompt: crie uma coluna chamada status  para o dataframe df_sp onde caso a sutuação cadastral esteja baixada, o status deve estar como encerrada, caso esteja ativa verifique se a coluna desenquadrado está com o valor S, e caso positivo, atribua o valor desenquadrado, e caso contrário como ativo. para as demais situações cadastrais atribuir o valor empresa irregular

# Crie a coluna 'status' com valores iniciais como 'empresa irregular'
df_sp['status'] = 'empresa irregular'

# Atualize o status com base nas condições especificadas
df_sp.loc[df_sp['situacao_cadastral'] == 'baixada', 'status'] = 'encerrada'
df_sp.loc[(df_sp['situacao_cadastral'] == 'ativa') & (df_sp['desenquadrado'] == 'S'), 'status'] = 'desenquadrado'
df_sp.loc[(df_sp['situacao_cadastral'] == 'ativa') & (df_sp['desenquadrado'] == 'N'), 'status'] = 'ativo'

In [ ]:
df_sp[['periodo','centro_expandido','status']].value_counts()

In [ ]:
# prompt: Plote uma tabela usando essa informação df_sp[['periodo','centro_expandido','status']].value_counts() como base, mas usando os valores de período e centro_expandido como nomes de colunas. Troque N para Periferia e S para Centro Expandido na coluna centro_expandido.

import pandas as pd

# Assuming df_sp is already defined as in the provided code

# Create the table
table = df_sp[['periodo', 'centro_expandido', 'status']].value_counts().reset_index(name='count')

# Replace 'N' and 'S' in 'centro_expandido'
table['centro_expandido'] = table['centro_expandido'].replace({'N': 'Periferia', 'S': 'Centro Expandido'})

# Pivot the table
pivot_table = pd.pivot_table(table, values='count', index=['periodo', 'status'], columns='centro_expandido', aggfunc='sum', fill_value=0)

# Display the table
pivot_table

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Supondo que df_sp já esteja definido

# Criar a tabela de contagem
table = df_sp[['periodo', 'centro_expandido', 'status']].value_counts().reset_index(name='count')

# Substituir 'N' e 'S' em 'centro_expandido'
table['centro_expandido'] = table['centro_expandido'].replace({'N': 'Periferia', 'S': 'Centro Expandido'})

# Definir a ordem correta dos períodos
periodos_esperados = ['antes', 'pandemia', 'depois']
table['periodo'] = pd.Categorical(table['periodo'], categories=periodos_esperados, ordered=True)

# Pivotar a tabela
pivot_table = pd.pivot_table(
    table,
    values='count',
    index=['centro_expandido', 'periodo'],
    columns=['status'],
    aggfunc='sum',
    fill_value=0
).reset_index()

# Definir os cabeçalhos
headers = ['Região', 'Período', 'Ativo', 'Desenquadrado', 'Empresa Irregular', 'Encerrada']

# Preparar os valores das células
cells_values = [pivot_table[col].tolist() for col in pivot_table.columns]

# Criar a tabela no Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=headers,
        align='center',
        line_color='black',
        fill_color='lightblue',
        font=dict(color='black', size=12),
        height=30
    ),
    cells=dict(
        values=cells_values,
        align='center',
        line_color='black',
        fill=dict(color=['white', 'lightgray']),
        font=dict(color='black', size=11),
        height=25
    )
)])

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Supondo que df_sp já esteja definido

# Agrupar os dados por 'periodo' e 'centro_expandido'
grouped = df_sp.groupby(['periodo', 'centro_expandido'])

# Calcular a média de 'meses_duracao' e 'meses_abandono' e a contagem de valores não-nulos
result = grouped[['meses_duracao', 'meses_abandono']].agg(['mean', 'count'])

# Renomear colunas para maior clareza
result.columns = ['tempo_medio_baixa', 'empresas_baixadas', 'tempo_medio_inaptidao', 'empresas_inaptas']

# Calcular a contagem total de registros para cada grupo (incluindo valores nulos)
total_counts = grouped.size().rename('total_registros')

# Combinar os resultados com a contagem total
result = pd.concat([result, total_counts], axis=1)

# Calcular o percentual de registros
result['percentual_baixadas'] = (result['empresas_baixadas'] / result['total_registros']) * 100
result['percentual_inaptas'] = (result['empresas_inaptas'] / result['total_registros']) * 100

# Arredondar os valores para uma casa decimal
result = result.round({'tempo_medio_baixa': 1, 'tempo_medio_inaptidao': 1, 'percentual_baixadas': 1, 'percentual_inaptas': 1})

# Resetar o índice para a tabela plotly
result = result.reset_index()

# Substituir 'N' e 'S' em 'centro_expandido'
result['centro_expandido'] = result['centro_expandido'].replace({'N': 'Periferia', 'S': 'Centro Expandido'})

# Definir a ordem correta dos períodos
periodos_esperados = ['antes', 'pandemia', 'depois']
result['periodo'] = pd.Categorical(result['periodo'], categories=periodos_esperados, ordered=True)
result = result.sort_values(by=['centro_expandido', 'periodo'])

# Definir os cabeçalhos
headers = ['Região', 'Período', 'Tempo Médio de Baixa (meses)', 'Empresas Baixadas', 'Tempo Médio de Inaptidão (meses)', 'Empresas Inaptas', 'Total de Registros', 'Percentual de Empresas Baixadas (%)', 'Percentual de Empresas Inaptas (%)']

# Preparar os valores das células
cells_values = [
    result['centro_expandido'].tolist(),
    result['periodo'].tolist(),
    result['tempo_medio_baixa'].tolist(),
    result['empresas_baixadas'].tolist(),
    result['tempo_medio_inaptidao'].tolist(),
    result['empresas_inaptas'].tolist(),
    result['total_registros'].tolist(),
    result['percentual_baixadas'].tolist(),
    result['percentual_inaptas'].tolist()
]

# Criar a tabela no Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=headers,
        align='center',
        line_color='black',
        fill_color='lightblue',
        font=dict(color='black', size=12),
        height=30
    ),
    cells=dict(
        values=cells_values,
        align='center',
        line_color='black',
        fill=dict(color=['white', 'lightgray']),
        font=dict(color='black', size=11),
        height=25
    )
)])

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Supondo que df_sp já esteja definido

# Criar a tabela de contagem
table = df_sp[['periodo', 'centro_expandido', 'status']].value_counts().reset_index(name='count')

# Substituir 'N' e 'S' em 'centro_expandido'
table['centro_expandido'] = table['centro_expandido'].replace({'N': 'Periferia', 'S': 'Centro Expandido'})

# Definir a ordem correta dos períodos
periodos_esperados = ['antes', 'pandemia', 'depois']
table['periodo'] = pd.Categorical(table['periodo'], categories=periodos_esperados, ordered=True)

# Pivotar a tabela
pivot_table = pd.pivot_table(
    table,
    values='count',
    index=['centro_expandido', 'periodo'],
    columns=['status'],
    aggfunc='sum',
    fill_value=0
).reset_index()

# Calcular os valores percentuais e arredondar para uma casa decimal
pivot_table_percentage = pivot_table.copy()
for periodo in periodos_esperados:
    for centro in ['Centro Expandido', 'Periferia']:
        mask = (pivot_table_percentage['periodo'] == periodo) & (pivot_table_percentage['centro_expandido'] == centro)
        total = pivot_table.loc[mask, pivot_table.columns[2:]].sum(axis=1).values[0]
        if total > 0:
            pivot_table_percentage.loc[mask, pivot_table.columns[2:]] = (pivot_table_percentage.loc[mask, pivot_table.columns[2:]] / total * 100).round(1)

# Definir os cabeçalhos
headers = ['Região', 'Período', 'Ativo (%)', 'Desenquadrado (%)', 'Empresa Irregular (%)', 'Encerrada (%)']

# Preparar os valores das células
cells_values = [pivot_table_percentage[col].tolist() for col in pivot_table_percentage.columns]

# Criar a tabela no Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=headers,
        align='center',
        line_color='black',
        fill_color='lightblue',
        font=dict(color='black', size=12),
        height=30
    ),
    cells=dict(
        values=cells_values,
        align='center',
        line_color='black',
        fill=dict(color=['white', 'lightgray']),
        font=dict(color='black', size=11),
        height=25
    )
)])

fig.show()


### Ramos de atividade

In [ ]:
import pandas as pd
import zipfile

# Carregar os dados do CNAE
try:
    cnae_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/estudo_mei/cnaes.zip', sep=';', header=None, encoding='latin1', names=['codigo', 'descricao'], compression='zip', dtype={'codigo': str}, on_bad_lines='skip')
except FileNotFoundError:
    print("Error: 'cnaes.zip' not found. Please check the file path.")
    cnae_df = pd.DataFrame()  # Create an empty DataFrame if the file isn't found
except zipfile.BadZipFile:
    print("Error: Invalid zip file. Please ensure the file is a valid zip archive.")
    cnae_df = pd.DataFrame()

if not cnae_df.empty:
    # Garantir que a coluna 'codigo' exista e seja do tipo string.
    if 'codigo' not in cnae_df.columns:
        print("Error: 'codigo' column not found in the provided file.")
    else:
        cnae_df['codigo'] = cnae_df['codigo'].astype(str)

        # Função para extrair a subclasse
        def get_subclass(cnae_code):
            try:
                return cnae_code[:2]
            except:
                return None

        # Aplicar a função para criar a coluna 'subclass'
        cnae_df['subclass'] = cnae_df['codigo'].apply(get_subclass)

        # Dicionário de mapeamento das subclasses para as denominações
        subclass_mapping = {
            '01': 'Agricultura e Pecuária e extrativismo',
            '02': 'Agricultura e Pecuária e extrativismo',
            '03': 'Agricultura e Pecuária e extrativismo',
            '05': 'Agricultura e Pecuária e extrativismo',
            '06': 'Agricultura e Pecuária e extrativismo',
            '07': 'Agricultura e Pecuária e extrativismo',
            '08': 'Agricultura e Pecuária e extrativismo',
            '09': 'Agricultura e Pecuária e extrativismo',
            '10': 'Processos Fabris',
            '11': 'Processos Fabris',
            '12': 'Processos Fabris',
            '13': 'Processos Fabris',
            '14': 'Processos Fabris',
            '15': 'Processos Fabris',
            '16': 'Processos Fabris',
            '17': 'Processos Fabris',
            '18': 'Processos Fabris',
            '19': 'Processos Fabris',
            '20': 'Processos Fabris',
            '21': 'Processos Fabris',
            '22': 'Processos Fabris',
            '23': 'Processos Fabris',
            '24': 'Processos Fabris',
            '25': 'Processos Fabris',
            '26': 'Processos Fabris',
            '27': 'Processos Fabris',
            '28': 'Processos Fabris',
            '29': 'Processos Fabris',
            '30': 'Processos Fabris',
            '31': 'Processos Fabris',
            '32': 'Processos Fabris',
            '33': 'Processos Fabris',
            '35': 'Eletricidade e Gás',
            '36': 'Água e Esgoto',
            '37': 'Água e Esgoto',
            '38': 'Água e Esgoto',
            '39': 'Água e Esgoto',
            '41': 'Construção',
            '42': 'Construção',
            '43': 'Construção',
            '45': 'Comércio',
            '46': 'Comércio',
            '47': 'Comércio',
            '49': 'Transporte Terrestre',
            '50': 'Transporte Aquaviário',
            '51': 'Transporte Aéreo',
            '52': 'Logística',
            '53': 'Atividades de Entrega',
            '55': 'Alojamento',
            '56': 'Alimentação',
            '58': 'Informação e Comunicação',
            '59': 'Produção de Vídeo e Som',
            '60': 'Informação e Comunicação',
            '61': 'Informação e Comunicação',
            '62': 'Serviços de TI',
            '63': 'Informação e Comunicação',
            '64': 'Atividades Financeiras, de Seguros e Serviços Relacionados',
            '65': 'Atividades Financeiras, de Seguros e Serviços Relacionados',
            '66': 'Atividades Financeiras, de Seguros 3 Serviços Relacionados',
            '68': 'Imobiliárias',
            '69': 'Atividades Profissionais, Científicas e Técnicas',
            '70': 'Atividades Profissionais, Científicas e Técnicas',
            '71': 'Atividades Profissionais, Científicas e Técnicas',
            '72': 'Atividades Profissionais, Científicas e Técnicas',
            '73': 'Publicidade',
            '74': 'Atividades Profissionais, Científicas e Técnicas',
            '75': 'Veterinário',
            '77': 'Atividades Administrativas E Serviços Complementares',
            '78': 'Atividades Administrativas E Serviços Complementares',
            '79': 'Atividades Administrativas E Serviços Complementares',
            '80': 'Atividades Administrativas E Serviços Complementares',
            '81': 'Atividades Administrativas E Serviços Complementares',
            '82': 'Atividades Administrativas E Serviços Complementares',
            '84': 'Servícos Públicos',
            '85': 'Educação',
            '86': 'Serviços de Saúde e Sociais',
            '87': 'Serviços de Saúde e Sociais',
            '88': 'Serviços de Saúde e Sociais',
            '90': 'Cultura',
            '91': 'Cultura, Esporte e Entretenimento',
            '92': 'Jogos de Azar e Apostas',
            '93': 'Esporte e Entretenimento',
            '94': 'Outros Serviços',
            '95': 'Manutenção de Computadores e eletrônicos',
            '96': 'Outros Serviços Pessoais',
            '97': 'Serviços Domésticos',
            '99': 'Órgãos Internacionais'
        }


        # Mapear as subclasses para as denominações
        cnae_df['denominacao'] = cnae_df['subclass'].map(subclass_mapping)

        # Exibir o DataFrame resultante
        cnae_df = cnae_df[['descricao', 'denominacao']]
        print(cnae_df.head())
        cnae_df.to_csv('/content/drive/MyDrive/Colab Notebooks/estudo_mei/cnaes_mapeados.zip', index=False, compression='zip')


In [ ]:
# prompt: criar a coluna 'subclasse_cnae' para o dataframe df_sp onde a partir do das correspondências entre cnae_fiscal_principal de df_sp e descricao de cnae_df retornar denominacao  de cnae_df

# Merge the dataframes based on the 'cnae_fiscal_principal' and 'descricao' columns
df_sp = pd.merge(df_sp, cnae_df, left_on='cnae_fiscal_principal', right_on='descricao', how='left')

# Rename the 'denominacao' column to 'subclasse_cnae'
df_sp = df_sp.rename(columns={'denominacao': 'subclasse_cnae'})

# Display the first few rows of the updated dataframe to verify the changes
print(df_sp[['cnae_fiscal_principal', 'subclasse_cnae']].head())

In [ ]:
# prompt: Plotar um gráfico temporal de áreal com valores acumulados de subclasse_cnae do dataframe df_sp para cadasttros com a situação ativa, com o fundo separado em antes, pandemia e depois.

import plotly.express as px

# Filtrar para estabelecimentos ativos
df_ativos_sp = df_sp[df_sp['situacao_cadastral'] == 'ativa']

# Agrupar por data e subclasse CNAE, somando os valores
df_agrupado_sp = df_ativos_sp.groupby([pd.Grouper(key='data_inicio', freq='ME'), 'subclasse_cnae'])['cnpj_basico'].count().reset_index()
df_agrupado_sp = df_agrupado_sp.rename(columns={'cnpj_basico': 'quantidade'})

# Calcular a soma acumulada da quantidade de CNPJs por subclasse CNAE
df_agrupado_sp['quantidade_acumulada'] = df_agrupado_sp.groupby('subclasse_cnae')['quantidade'].cumsum()

# Criar o gráfico de área
fig = px.area(df_agrupado_sp,
              x='data_inicio',
              y='quantidade_acumulada',
              color='subclasse_cnae',
              title='Valores Acumulados de Subclasse CNAE (Estabelecimentos Ativos)',
              labels={'data_inicio': 'Data de Início', 'quantidade_acumulada': 'Quantidade Acumulada', 'subclasse_cnae': 'Subclasse CNAE'})

# Adicionar áreas de fundo para as épocas
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")


fig.show()

In [ ]:
# prompt: Refaça o gráfico da célula anterior trocando a quantidade acumulada de registros por quantidade mensal e a área por linha. Utilize a média móvel de 3 meses para atenuar eventuais sazonalidades.

# Calcular a média móvel de 3 meses
df_agrupado_sp['quantidade_mm3'] = df_agrupado_sp.groupby('subclasse_cnae')['quantidade'].rolling(window=3, min_periods=1).mean().reset_index(0,drop=True)

# Criar o gráfico de linha com a média móvel
fig = px.line(df_agrupado_sp,
              x='data_inicio',
              y='quantidade_mm3',
              color='subclasse_cnae',
              title='Quantidade Mensal de MEIs por Subclasse CNAE (Média Móvel de 3 Meses)',
              labels={'data_inicio': 'Data de Início', 'quantidade_mm3': 'Quantidade (Média Móvel 3 Meses)', 'subclasse_cnae': 'Subclasse CNAE'})

# Adicionar áreas de fundo para as épocas
fig.add_vrect(x0='2020-02-03', x1='2022-05-22',
             fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pandemia", annotation_position="top left")

fig.add_vrect(x0='2018-02-03', x1='2020-02-02',
             fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pré-Pandemia", annotation_position="top left")

fig.add_vrect(x0='2022-05-23', x1='2024-05-22',
             fillcolor="pink", opacity=0.2, layer="below", line_width=0,
             annotation_text="Pós-Pandemia", annotation_position="top left")

fig.show()

### Estudo de bairros

### Roteiro

Projeto da pós-graduação

- Dados a apurar:

1. Crescimento geral
  1. Número de cadastros
  2. Evolução por situação (acumulado)
  3. Mapas (antes, pandemia, depois)

2. Crescimento (centro expandido x periferia)
  1. Número de cadastros
  2. Evolução por situação (acumulado)
  3. Tabelas:
    1. Quantidade de bairros com mais de cem e mil MEI's ativos, por macrorregião e período;
    2. Relação dos dez bairros com mais MEI's criados, por período;
    3. Taxa de sobrevivência (% de MEI's baixados), por macrorregião e período;
    4. Taxa de inaptidão (% de MEI's inaptos), por macrorregião e período;

- Conclusões:

1. Houve aumento da capilaridade de MEI's?
2. Houve alteração no perfil de ramos de atividades?
3. Mudou a resiliência (baixa e inaptidão)?
4. Que pontos a identificar e quais propostas e sugestões de melhoria?

### Crescimento geral - Número de cadastros

In [ ]:
# prompt: Faça um gráfico de área para a quantidade acumulada de registros mês a mês de criação do MEI's na cidade de são paulo, ilustrando os períodos pré, pandemia e depois, separando por periferia e centro expandido

import plotly.express as px

# Assuming df_sp is your DataFrame and it has columns:
# 'data_inicio' (datetime), 'centro_expandido' (categorical: 'S' or 'N'),
# 'quantidade' (numeric - number of MEI registrations)

# Create a copy of your DataFrame to avoid modifying the original
df_plot = df_sp.copy()

# Convert 'data_inicio' to datetime if not already
df_plot['data_inicio'] = pd.to_datetime(df_plot['data_inicio'])

# Group data and calculate the cumulative sum
df_plot = df_plot.groupby(['data_inicio', 'centro_expandido'])['cnpj_basico'].count().reset_index()
df_plot = df_plot.rename(columns={'cnpj_basico':'quantidade'})
df_plot['quantidade_acumulada'] = df_plot.groupby('centro_expandido')['quantidade'].cumsum()

# Create the area chart
fig = px.area(df_plot,
              x='data_inicio',
              y='quantidade_acumulada',
              color='centro_expandido',
              title='Quantidade Acumulada de Registros de MEIs em São Paulo',
              labels={'data_inicio': 'Data de Início',
                      'quantidade_acumulada': 'Quantidade Acumulada de MEIs',
                      'centro_expandido': 'Região'})

fig.for_each_trace(lambda t: t.update(name={'S': 'Centro Expandido', 'N': 'Periferia'}[t.name]))

# Add vertical rectangles for the periods
fig.add_vrect(x0=pd.Timestamp('2020-02-01'), x1=pd.Timestamp('2022-05-31'),
             annotation_text="Período da Pandemia", annotation_position="top left",
             fillcolor="lightgreen", opacity=0.25, line_width=0)
fig.add_vrect(x0=df_plot['data_inicio'].min(), x1=pd.Timestamp('2020-01-31'),
              annotation_text="Período Pré-Pandemia", annotation_position="top left",
             fillcolor="lightblue", opacity=0.25, line_width=0)

fig.add_vrect(x0=pd.Timestamp('2022-06-01'), x1=df_plot['data_inicio'].max(),
              annotation_text="Período Pós-Pandemia", annotation_position="top left",
             fillcolor="pink", opacity=0.25, line_width=0)

fig.show()

In [ ]:
df_estabelecimentos = restore_dataframe(f'{DIR_DADOS}/estabelecimentos_mei.zip', filtros=f"municipio_nome == 'SAO PAULO' & uf == 'SP'")

In [ ]:
municipio, uf = 'SAO PAULO', 'SP'
locais_sp = logradouros_IBGE(municipio, uf)
df_cidade = pd.merge(df_estabelecimentos, locais_sp, on=['logradouro', 'bairro'], how='inner')
df_com_coordenadas = df_cidade.dropna(subset=['latitude', 'longitude'])
df_enderecos = df_com_coordenadas.groupby('bairro').agg({
        'latitude': 'mean',
        'longitude': 'mean'
}).reset_index()

In [ ]:
df_enderecos

In [ ]:
bairros_contidos = extrair_bairros_kml(f'{DIR_DADOS}/rodizio_sp.kml', df_enderecos)

In [ ]:
df_cidade['centro_expandido'] = 'N'  # Inicializa a coluna com 'N'
df_cidade.loc[df_cidade['bairro'].isin(bairros_contidos['bairro']), 'centro_expandido'] = 'S'
df_sp_antes = df_cidade[df_cidade['data_inicio'] < '2018-02-03']
backup_dataframe(df_sp_antes, f'{DIR_DADOS}/df_sp_antes.zip')

In [ ]:
df_sp_antes[['centro_expandido','situacao_cadastral']].value_counts()

In [ ]:
# prompt: refaça o gráfico anterior adicionando os valores anteriores apurados anteriores a todos os períodos do gráfico em quantidade acumulada de registros (255898 para N e 84822 para S na coluna centro_expandido)

# Assuming df_plot is the DataFrame used for the previous area chart
# and it has columns: 'data_inicio', 'centro_expandido', 'quantidade', 'quantidade_acumulada'

# Add the initial values for each 'centro_expandido'
initial_values = {'S': 84822, 'N': 255898}
for centro, value in initial_values.items():
    # Create a new row for the initial value
    new_row = pd.DataFrame({'data_inicio': df_plot['data_inicio'].min() - pd.Timedelta(days=1),
                            'centro_expandido': centro,
                            'quantidade': value,
                            'quantidade_acumulada': value}, index=[0])
    # Concatenate with the existing DataFrame
    df_plot = pd.concat([new_row, df_plot], ignore_index=True)

# Correct cumulative sum after adding initial values
df_plot['quantidade_acumulada'] = df_plot.groupby('centro_expandido')['quantidade'].cumsum()

# Create the area chart (using the updated df_plot)
fig = px.area(df_plot,
              x='data_inicio',
              y='quantidade_acumulada',
              color='centro_expandido',
              title='Quantidade Acumulada de Registros de MEIs em São Paulo por macrorregião',
              labels={'data_inicio': 'Data de Início',
                      'quantidade_acumulada': 'Quantidade Acumulada de MEIs',
                      'centro_expandido': 'Região'})

fig.for_each_trace(lambda t: t.update(name={'S': 'Centro Expandido', 'N': 'Periferia'}[t.name]))

# Add vertical rectangles for the periods
fig.add_vrect(x0=pd.Timestamp('2020-02-01'), x1=pd.Timestamp('2022-05-31'),
             annotation_text="Período da Pandemia", annotation_position="top left",
             fillcolor="lightgreen", opacity=0.25, line_width=0)
fig.add_vrect(x0=df_plot['data_inicio'].min(), x1=pd.Timestamp('2020-01-31'),
              annotation_text="Período Pré-Pandemia", annotation_position="top left",
             fillcolor="lightblue", opacity=0.25, line_width=0)

fig.add_vrect(x0=pd.Timestamp('2022-06-01'), x1=df_plot['data_inicio'].max(),
              annotation_text="Período Pós-Pandemia", annotation_position="top left",
             fillcolor="pink", opacity=0.25, line_width=0)

fig.show()

### Crescimento Geral - Evolução por situação

In [ ]:
df_sp_antes[['centro_expandido','situacao_cadastral']].value_counts()

In [ ]:
import pandas as pd
import plotly.express as px

# Converta a coluna 'data_situacao_cadastral' para o tipo datetime
df_merged['data_situacao_cadastral'] = pd.to_datetime(df_merged['data_situacao_cadastral'])

# Crie um dataframe com todas as combinações possíveis de datas, centro_expandido e situacao_cadastral
all_dates = pd.date_range(start=df_merged['data_situacao_cadastral'].min(), end=df_merged['data_situacao_cadastral'].max(), freq='ME')
all_combinations = pd.MultiIndex.from_product([all_dates, df_merged['centro_expandido'].unique(), df_merged['situacao_cadastral'].unique()], names=['data_situacao_cadastral', 'centro_expandido', 'situacao_cadastral'])
df_all_combinations = pd.DataFrame(index=all_combinations).reset_index()
all_combinations_t = pd.MultiIndex.from_product([all_dates, df_merged['situacao_cadastral'].unique()], names=['data_situacao_cadastral', 'situacao_cadastral'])
df_all_combinations_t = pd.DataFrame(index=all_combinations_t).reset_index()

# Agrupe os dados mensalmente pelo número de registros por cada situação cadastral, divididos pela coluna 'centro_expandido'
df_agrupado = df_merged.groupby([pd.Grouper(key='data_situacao_cadastral', freq='ME'), 'centro_expandido', 'situacao_cadastral']).size().reset_index(name='total_registros')
df_agrupado_total = df_merged.groupby([pd.Grouper(key='data_situacao_cadastral', freq='ME'),'situacao_cadastral']).size().reset_index(name='total_registros')

# Mescle com o dataframe de todas as combinações possíveis para preencher os meses sem dados com zeros
df_agrupado = df_all_combinations.merge(df_agrupado, on=['data_situacao_cadastral', 'centro_expandido', 'situacao_cadastral'], how='left').fillna(0)
df_agrupado_total = df_all_combinations_t.merge(df_agrupado_total, on=['data_situacao_cadastral', 'situacao_cadastral'], how='left').fillna(0)

# Calcule o total acumulado de registros
df_agrupado['total_acumulado'] = df_agrupado.groupby(['centro_expandido', 'situacao_cadastral'])['total_registros'].cumsum()
df_agrupado_total['total_acumulado'] = df_agrupado_total.groupby(['situacao_cadastral'])['total_registros'].cumsum()

df_sp_antes_init = df_sp_antes.groupby(['data_inicio', 'centro_expandido', 'situacao_cadastral']).size().reset_index(name='count')
# Atualize o valor inicial para o período anterior contido no dataframe df_sp_antes
for index, row in df_sp_antes_init.iterrows():
    mask = (df_agrupado['centro_expandido'] == row['centro_expandido']) & (df_agrupado['situacao_cadastral'] == row['situacao_cadastral'])
    df_agrupado.loc[mask, 'total_acumulado'] += row['count']

# Recalcule o total acumulado de registros para a cidade de São Paulo somando os valores de centro expandido e periferia
df_total_sp_corrigido = df_agrupado.groupby(['data_situacao_cadastral', 'situacao_cadastral'])['total_acumulado'].sum().reset_index()


In [ ]:

# Crie um gráfico de área preenchida mensal para o Centro Expandido
fig_total_sp_corrigido = px.area(df_total_sp_corrigido,
                                 x='data_situacao_cadastral',
                                 y='total_acumulado',
                                 color='situacao_cadastral',
                                 title='Total Acumulado de Situações Cadastrais por Mês (Cidade de São Paulo)',
                                 labels={'total_acumulado': 'Total Acumulado', 'data_situacao_cadastral': 'Mês', 'situacao_cadastral': 'Situação Cadastral'})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig_total_sp_corrigido.add_vrect(x0='2020-02-03', x1='2022-05-22',
                                 fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
                                 annotation_text="Pandemia", annotation_position="top left")

fig_total_sp_corrigido.add_vrect(x0='2018-02-03', x1='2020-02-02',
                                 fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
                                 annotation_text="Pré-Pandemia", annotation_position="top left")

fig_total_sp_corrigido.add_vrect(x0='2022-05-23', x1='2024-05-22',
                                 fillcolor="pink", opacity=0.2, layer="below", line_width=0,
                                 annotation_text="Pós-Pandemia", annotation_position="top left")

# fig_total_sp_corrigido.add_annotation(
#     text='Fonte: Receita Federal, compilado pelo Autor.',
#     xref='paper', yref='paper',
#     x=0, y=0.05,
#     showarrow=False,
#     font=dict(size=12)
# )

fig_total_sp_corrigido.show()

# Crie um gráfico de área preenchida mensal para o Centro Expandido
df_centro_expandido = df_agrupado[df_agrupado['centro_expandido'] == 'S']
fig_centro_expandido = px.area(df_centro_expandido,
                               x='data_situacao_cadastral',
                               y='total_acumulado',
                               color='situacao_cadastral',
                               title='Total Acumulado de Situações Cadastrais por Mês (Centro Expandido)',
                               labels={'total_acumulado': 'Total Acumulado', 'data_situacao_cadastral': 'Mês', 'situacao_cadastral': 'Situação Cadastral'})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig_centro_expandido.add_vrect(x0='2020-02-03', x1='2022-05-22',
                               fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
                               annotation_text="Pandemia", annotation_position="top left")

fig_centro_expandido.add_vrect(x0='2018-02-03', x1='2020-02-02',
                               fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
                               annotation_text="Pré-Pandemia", annotation_position="top left")

fig_centro_expandido.add_vrect(x0='2022-05-23', x1='2024-05-22',
                               fillcolor="pink", opacity=0.2, layer="below", line_width=0,
                               annotation_text="Pós-Pandemia", annotation_position="top left")

fig_centro_expandido.add_annotation(
    text='Fonte: Receita Federal, compilado pelo Autor.',
    xref='paper', yref='paper',
    x=0, y=0.05,
    showarrow=False,
    font=dict(size=12)
)

fig_centro_expandido.show()

# Crie um gráfico de área preenchida mensal para a Periferia
df_periferia = df_agrupado[df_agrupado['centro_expandido'] == 'N']
fig_periferia = px.area(df_periferia,
                        x='data_situacao_cadastral',
                        y='total_acumulado',
                        color='situacao_cadastral',
                        title='Total Acumulado de Situações Cadastrais por Mês (Periferia)',
                        labels={'total_acumulado': 'Total Acumulado', 'data_situacao_cadastral': 'Mês', 'situacao_cadastral': 'Situação Cadastral'})

# Adicione uma área de fundo para diferenciar por cor os períodos anterior e posterior a pandemia
fig_periferia.add_vrect(x0='2020-02-03', x1='2022-05-22',
                        fillcolor="lightgreen", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pandemia", annotation_position="top left")

fig_periferia.add_vrect(x0='2018-02-03', x1='2020-02-02',
                        fillcolor="lightblue", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pré-Pandemia", annotation_position="top left")

fig_periferia.add_vrect(x0='2022-05-23', x1='2024-05-22',
                        fillcolor="pink", opacity=0.2, layer="below", line_width=0,
                        annotation_text="Pós-Pandemia", annotation_position="top left")

fig_periferia.add_annotation(
    text='Fonte: Receita Federal, compilado pelo Autor.',
    xref='paper', yref='paper',
    x=0, y=0.05,
    showarrow=False,
    font=dict(size=12)
)

fig_periferia.show()

### Crescimento Geral - Mapas e capilaridade

In [ ]:
# prompt: vamos usar a função plotar_mapa_estabelecimentos(df_merged, 'São Paulo', 'mapa_mei_sp') para gerar três mapas de acordo com os períodos e pré-pandemia, pandemia e pós-pandemia apenas com os registros com situacao_cadastral = ativa

# Filtrar o DataFrame para incluir apenas registros com 'situacao_cadastral' igual a 'ativa'
df_ativos = df_merged[df_merged['situacao_cadastral'] == 'ativa']

# Gerar o mapa para o período pré-pandemia
titulo1 = "Mapa de MEI's Ativos em São Paulo (Pré-Pandemia)"
nota1 = 'Estabelecimentos MEI criados na cidade de São Paulo entre 03/02/2018 e 02/02/2020.'
df_pre_pandemia = df_ativos[df_ativos['periodo'] == 'antes']
fig_pre_pandemia = plotar_mapa_estabelecimentos(df_pre_pandemia, 'São Paulo', 'mapa_mei_sp_pre_pandemia', titulo=titulo1, nota=nota1)
fig_pre_pandemia.show()

# Gerar o mapa para o período da pandemia
titulo2 = "Mapa de MEI's Ativos em São Paulo (Pandemia)"
nota2 = 'Estabelecimentos MEI criados na cidade de São Paulo entre 03/02/2020 e 22/05/2022.'
df_pandemia = df_ativos[df_ativos['periodo'] == 'pandemia']
fig_pandemia = plotar_mapa_estabelecimentos(df_pandemia, 'São Paulo', 'mapa_mei_sp_pandemia', titulo=titulo2, nota=nota2)
fig_pandemia.show()

# Gerar o mapa para o período pós-pandemia
titulo3 = "Mapa de MEI's Ativos em São Paulo (Pós-Pandemia)"
nota3 = 'Estabelecimentos MEI criados na cidade de São Paulo entre 23/05/2022 e 23/05/2024.'
df_pos_pandemia = df_ativos[df_ativos['periodo'] == 'depois']
fig_pos_pandemia = plotar_mapa_estabelecimentos(df_pos_pandemia, 'São Paulo', 'mapa_mei_sp_pos_pandemia', titulo=titulo3, nota=nota3)
fig_pos_pandemia.show()

### Crescimento por região - Quantidade de bairros com mais de cem e mil MEI's ativos, por macrorregião e período



In [ ]:
import plotly.graph_objects as go

def plot_table(df_sp):
    # Agrupa os dados por período, centro expandido e conta a quantidade de bairros com mais de 500 e 1000 MEIs
    table_data = df_sp.groupby(['periodo', 'centro_expandido', 'bairro'])['cnpj_basico'].count().reset_index()
    table_data = table_data[table_data['cnpj_basico'] > 0]

    table_data['mais_de_500_MEIs'] = table_data['cnpj_basico'] > 500
    table_data['mais_de_1000_MEIs'] = table_data['cnpj_basico'] > 1000

    # Cria a tabela
    periodos = ['antes', 'pandemia', 'depois']
    centros = ['S', 'N']

    header = dict(values=['Período', 'Região', 'Total de Bairros com MEI\'s', 'Bairros com +500 MEI\'s', 'Bairros com +1000 MEI\'s'],
                  align='left')

    cells_values = [[], [], [], [], []]

    for periodo in periodos:
        for centro in centros:
            cells_values[0].append(periodo)
            cells_values[1].append('Centro Expandido' if centro == 'S' else 'Periferia')
            total_bairros = table_data[(table_data['periodo'] == periodo) & (table_data['centro_expandido'] == centro)].shape[0]
            bairros_500 = table_data[(table_data['periodo'] == periodo) & (table_data['centro_expandido'] == centro) & (table_data['mais_de_500_MEIs'])].shape[0]
            bairros_1000 = table_data[(table_data['periodo'] == periodo) & (table_data['centro_expandido'] == centro) & (table_data['mais_de_1000_MEIs'])].shape[0]
            cells_values[2].append(total_bairros)
            cells_values[3].append(bairros_500)
            cells_values[4].append(bairros_1000)

    cells = dict(values=cells_values,
                 align='left',
                 fill_color='paleturquoise',
                 font=dict(color='black'))

    fig = go.Figure(data=[go.Table(header=header, cells=cells)])

    fig.update_layout(
    title='Contagem de bairros com MEI\'s por período e região na cidade de São Paulo',
    annotations=[dict(text='Fonte: Receita Federal do Brasil, 2024. Compilado pelo autor.',
                      x=0, y=0.5,
                      xref='paper', yref='paper',
                      showarrow=False,
                      xanchor='left')]
    )
    fig.show()

# Exemplo de uso (substitua 'seu_dataframe.csv' pelo caminho real para seu arquivo)
# df_sp = pd.read_csv('seu_dataframe.csv')
plot_table(df_sp)


### Crescimento por região - Relação dos 10 bairros mais ativos por região

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_table(df_sp):
    # Cria os dados da tabela
    antes_top10 = df_sp[df_sp['periodo'] == 'antes'].groupby('bairro')['cnpj_basico'].count().nlargest(10).index.tolist()
    pandemia_top10 = df_sp[df_sp['periodo'] == 'pandemia'].groupby('bairro')['cnpj_basico'].count().nlargest(10).index.tolist()
    depois_top10 = df_sp[df_sp['periodo'] == 'depois'].groupby('bairro')['cnpj_basico'].count().nlargest(10).index.tolist()

    header = dict(values=['Antes da Pandemia', 'Período da Pandemia', 'Após a Pandemia'],
                  align='left')

    # Determina as cores das células
    antes_colors = ['blue' if df_sp[(df_sp['bairro'] == bairro) & (df_sp['periodo'] == 'antes')]['centro_expandido'].iloc[0] == 'S' else 'red' for bairro in antes_top10]
    pandemia_colors = ['blue' if df_sp[(df_sp['bairro'] == bairro) & (df_sp['periodo'] == 'pandemia')]['centro_expandido'].iloc[0] == 'S' else 'red' for bairro in pandemia_top10]
    depois_colors = ['blue' if df_sp[(df_sp['bairro'] == bairro) & (df_sp['periodo'] == 'depois')]['centro_expandido'].iloc[0] == 'S' else 'red' for bairro in depois_top10]

    cells = dict(values=[antes_top10, pandemia_top10, depois_top10],
                 align='left',
                 font=dict(color=[antes_colors, pandemia_colors, depois_colors]))

    fig = go.Figure(data=[go.Table(header=header, cells=cells)])

    # Adiciona título e nota de rodapé
    fig.update_layout(
        title='10 Bairros com mais MEI\'s ativos São Paulo por Período',
        annotations=[dict(text='Fonte: Receita Federal do Brasil, 2024. Compilado pelo autor.',
                          x=0, y=0.25,
                          xref='paper', yref='paper',
                          showarrow=False,
                          xanchor='left')]
    )

    fig.show()

# Exemplo de uso
# df_sp = pd.read_csv('seu_arquivo.csv')  # Carregue seu dataframe aqui
meis_ativos = df_sp[df_sp['situacao_cadastral'] == 'ativa']
plot_table(meis_ativos)


### Crescimento por região - Taxa de sobrevivência e inaptidão

In [ ]:
import plotly.graph_objects as go

def plot_survival_and_inaptitude_tables(result):
    # Create a copy of the DataFrame to avoid modifying the original
    result_copy = result.copy()

    # Replace 'S' and 'N' with 'Centro Expandido' and 'Periferia'
    result_copy['centro_expandido'] = result_copy['centro_expandido'].replace({'S': 'Centro Expandido', 'N': 'Periferia'})

    # Define periods
    periods = ['antes', 'pandemia', 'depois']

    # Function to create a combined table
    def create_combined_table(data, title, duration_col, percent_col):
        regions = data['centro_expandido'].unique()
        duration_values = [
            data[data['periodo'] == period][duration_col].round(1).astype(str).replace('nan', 'Não aplicável').tolist()
            for period in periods
        ]
        percent_values = [
            data[data['periodo'] == period][percent_col].round(1).astype(str).replace('nan', 'Não aplicável').tolist()
            for period in periods
        ]

        fig = go.Figure(data=[go.Table(
            header=dict(values=['Região'] + [f'Tempo Médio de {title} em Meses ({period})' for period in periods] +
                        [f'Percentual de {title} ({period})' for period in periods],
                        fill_color='paleturquoise',
                        align='left'),
            cells=dict(values=[regions] + duration_values + percent_values,
                       fill_color='lavender',
                       align='left'))
        ])
        titlefig = f'Tempo médio e percentual de {title} de MEI\'s na cidade de São Paulo por Região e Período'
        fig.update_layout(title=titlefig)
        fig.show()

    # Combined Survival Rate Table
    create_combined_table(result_copy, 'Baixa', 'meses_duracao_mean', 'meses_duracao_percent')

    # Combined Inaptitude Rate Table
    create_combined_table(result_copy, 'Inaptidão', 'meses_abandono_mean', 'meses_abandono_percent')

# Example usage (replace with your actual DataFrame)
plot_survival_and_inaptitude_tables(result.reset_index())

## Análise de principais atividades

In [ ]:
import plotly.graph_objects as go

def plot_top_cnae_table(df):
    # Group data by period, expanded center, and main CNAE code, counting occurrences
    cnae_counts = df.groupby(['periodo', 'centro_expandido', 'cnae_fiscal_principal'])['cnpj_basico'].count().reset_index(name='count')

    # Sort values within each group to get the top 5 CNAEs
    cnae_counts = cnae_counts.sort_values(['periodo', 'centro_expandido', 'count'], ascending=[True, True, False])

    # Create the table data
    periods = ['antes', 'pandemia', 'depois']
    centers = cnae_counts['centro_expandido'].unique()

    # Initialize header and cells_values for the table
    header = dict(values=['Região'] + periods, align='left')
    cells_values = [[], [], [], []]

    for center in centers:
        # Replace 'N' and 'S' with 'Periferia' and 'Centro Expandido'
        region_name = 'Periferia' if center == 'N' else 'Centro Expandido'
        cells_values[0].append(region_name)  # Add center to the first column (Região)

        for period in periods:
            top_cnaes = cnae_counts[(cnae_counts['periodo'] == period) & (cnae_counts['centro_expandido'] == center)].head(5)
            cnae_list = top_cnaes['cnae_fiscal_principal'].tolist()
            # Ensure there are 5 entries
            if len(cnae_list) < 5:
                cnae_list.extend([''] * (5 - len(cnae_list)))
            # Create a numbered list with HTML line breaks
            numbered_cnae_list = [f"{i+1}. {cnae}<br>" for i, cnae in enumerate(cnae_list)]
            cells_values[periods.index(period) + 1].append(''.join(numbered_cnae_list))

    cells = dict(values=cells_values, align='left', fill_color='paleturquoise', height=30)

    fig = go.Figure(data=[go.Table(header=header, cells=cells, columnwidth=[40, 200, 200, 200])])

    fig.update_layout(title="Top 5 CNAE's por Região e Período")

    fig.show()

# Exemplo de uso (substitua pelo seu DataFrame)
plot_top_cnae_table(df_merged)
